In [10]:
from pathlib import Path
from tqdm import tqdm
from sklearn.decomposition import PCA

import math
import numpy as np
import pandas as pd
import os
import json
import pickle
import torch



ImportError: libGL.so.1: cannot open shared object file: No such file or directory

In [2]:
PATH = Path('data')
data = None
with open(PATH/'youcookii_annotations_trainval.json', 'rb') as f:
    data = json.load(f)
    
    

In [9]:
import clip

ImportError: libGL.so.1: cannot open shared object file: No such file or directory

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('/common/home/dm1487/loaded_models/clip/ViT-B-32.pt', device=device)


In [ ]:
image = preprocess(Image.open("CLIP.png")).unsqueeze(0).to(device)
# text = clip.tokenize(["a diagram", "a dog", "a cat"]).to(device)

In [8]:
!nvidia-smi

Sat Apr 23 21:59:49 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A4000    On   | 00000000:1A:00.0 Off |                 Off* |
| 41%   46C    P2    37W / 140W |   1124MiB / 16117MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA RTX A4000    On   | 00000000:1B:00.0 Off |                 Off* |
| 65%   

In [ ]:
# # text_data_file = PATH/'youcook_text.csv'
# # text_dataset = []

# # for _id, rec in tqdm(data['database'].items()):
# #     recipe_type = rec['recipe_type']
# #     subset = rec['subset']
# #     for ann in rec['annotations']:
# #         text_dataset.append([recipe_type, _id, subset, ann['id'], ann['segment'], ann['sentence']])
# # df = pd.DataFrame(text_dataset, columns=['recipe_type', 'key', 'subset', 'id', 'segment', 'sentence'])
# # df.to_csv(text_data_file, index=False)
# data['database']['ucky1nWb7LY']

In [ ]:
DATA_PATH = Path('/common/users/dm1487/YouCookII')
os.listdir(DATA_PATH/'features/feat_csv/train_frame_feat_csv/101/ucky1nWb7LY/0001')
DATA_DIR = DATA_PATH/'features/feat_csv/train_frame_feat_csv/101/ucky1nWb7LY/0001'
df = pd.read_csv(DATA_DIR/'resnet_34_feat_mscoco.csv', header=None)

image_features = np.array([df.iloc[i].tolist() for i in range(df.shape[0])])
image_features.shape

In [ ]:
text_data_file = PATH/'youcook_text.csv'
df_text = pd.read_csv(text_data_file)
df_text[df_text['key'] == 'ucky1nWb7LY'].index 

In [ ]:
# with open(PATH/'features_text_instructions/all_text_features.pkl', 'rb') as f:
#     text_features = pickle.load(f)
text_features = torch.load(PATH/'features_text_instructions/all_text_features.pkl')
# text_features[df_text[df_text['key'] == 'ucky1nWb7LY'].index].shape

In [ ]:
pca = PCA(512)
# print(text_features.cpu().numpy())
text_features_dr = torch.tensor(pca.fit_transform(text_features.cpu().numpy()))

In [ ]:
text_features_dr.size()

In [ ]:
mat = text_features_dr[df_text[df_text['key'] == 'ucky1nWb7LY'].index] @ image_features.T

In [ ]:
# torch.softmax(mat, dim = 0)
val, ind = torch.topk(mat, dim=1, k=10)
torch.sort(ind,dim=1)

In [ ]:
sorted(os.listdir(PATH/'feat_dat/train_frame_feat/102/0IuQKThr-pM/0001'))

np.loadtxt(PATH/'feat_dat/train_frame_feat/102/0IuQKThr-pM/0001/resnet_34_feat_mscoco.dat')

In [ ]:
sorted(os.listdir(PATH/'features'/'feat_csv/train_frame_feat_csv/101'))

In [ ]:
df = pd.read_csv(PATH/'features'/'feat_csv/train_frame_feat_csv/113/2vNPfc8LaTc/0010/resnet_34_feat_mscoco.csv', header=None)
df.shape

In [ ]:
# train_split = None
# with open(PATH/'splits/train_list.txt', 'r') as f:
#     train_split = f.readlines()
# train_split
os.listdir(PATH/'splits')

In [ ]:
df_linker = pd.read_csv(PATH/'splits'/'train_duration_totalframe.csv')
# df_linker[df_linker['vid_id'] == '2vNPfc8LaTc']
df_linker['I'] = (df_linker['total_frame']/500).apply(math.ceil)
df_linker['fps'] = (df_linker['total_frame']/df_linker['duration']).apply(math.ceil)
df_linker['spf'] = (1/df_linker['fps'])
df_linker.head()

In [ ]:
# df.set_index('key').join(df_linker.set_index('vid_id')).head(20)

In [ ]:
# df_linker['frames_needed'] = [None] * df_linker.shape[0]
frames_disected = []
for idx, row in df_linker.iterrows():
    frames_needed = []
    frames_repre = []
    m = 0
    while m < row['total_frame']:
        frames_needed.append(m)
        frames_repre.append(m * row['spf'])
        m += row['I']
#     print(frames_needed)
    frames_disected.append([row['vid_id'], frames_needed.copy(), frames_repre.copy()])
#     break
# df_linker.head()
# df_linker['frames_needed'] = df_linker['fps']/df_linker['I']

In [ ]:
df_segments = pd.DataFrame(frames_disected, columns=['vid_id', 'frames_recorded', 'segments']).set_index('vid_id').join(df.set_index('key'))
df_segments['segment'] = df_segments['segment'].apply(eval)
df_segments.head()

In [ ]:
frames_needed_for_extract = []
for idx, row in df_segments.iterrows():
    start, end = tuple(row['segment'])
    arr = np.array(row['segments'])
    arg_index = np.argwhere((arr >= start) & (arr < end))
    frames_needed_for_extract.append([idx, arg_index.flatten().tolist()])

df_extract = pd.DataFrame(frames_needed_for_extract, columns=['vid_id', 'extract'])

In [ ]:
df.set_index('key').join(df_extract.set_index('vid_id'))

In [ ]:
all_df = df.set_index('key').join(df_linker.set_index('vid_id'))

In [ ]:
all_df = all_df[all_df['subset'] == 'training']


In [ ]:
all_df['segment']